In [4]:
#导入必要的库
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn import preprocessing
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from math import sqrt
from keras.layers import *
from keras.models import *
from keras.optimizers import Adam
from sklearnex import patch_sklearn
patch_sklearn()

Using TensorFlow backend.
c:\Users\tangj15\AppData\Local\anaconda3\envs\py36_tf19\lib\site-packages\tensorflow\python\framework\dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
c:\Users\tangj15\AppData\Local\anaconda3\envs\py36_tf19\lib\site-packages\tensorflow\python\framework\dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
c:\Users\tangj15\AppData\Local\anaconda3\envs\py36_tf19\lib\site-packages\tensorflow\python\framework\dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np

In [5]:
# 创建映射字典
label_number = {"装": 1, "卸": -1}

# 定义一个函数，用于将Category列的文本转换为对应数值
# def label_to_number(row):
#     v = row["state"]
#     if v not in label_number:
#         return 0
#     return label_number[v]


# def number_to_label(row):
#     v = row["state"].strip()
#     display(v)
#     if v == 1:
#         return "装"
#     elif v == -1:
#         return "卸"
#     else:
#         return "-"


dfstate = pd.read_csv("./data/states.txt")
dfstate["start"] = dfstate.apply(
    lambda row: row["date"] + " " + row["start_time"], axis=1
)
dfstate["end"] = dfstate.apply(lambda row: row["date"] + " " + row["end_time"], axis=1)
dfstate["label"] = dfstate.pop("state").str.strip()
# 转为时间类型
dfstate[["start", "end"]] = dfstate[["start", "end"]].apply(pd.to_datetime)
# dfstate["statev"] = dfstate.apply(label_to_number, axis=1)
# dfstate['label'] = dfstate.apply(number_to_label,axis=1)
dfstate.to_csv("./data/states.csv")
dfstate.head(), dfstate.shape

(         date start_time  end_time               start                 end  \
 0  2023-12-17   10:44:49  10:48:59 2023-12-17 10:44:49 2023-12-17 10:48:59   
 1  2023-12-17   11:29:45  11:31:55 2023-12-17 11:29:45 2023-12-17 11:31:55   
 2  2023-12-17   12:23:56  12:29:44 2023-12-17 12:23:56 2023-12-17 12:29:44   
 3  2023-12-17   12:55:08  12:56:28 2023-12-17 12:55:08 2023-12-17 12:56:28   
 4  2023-12-17   13:45:02  13:45:42 2023-12-17 13:45:02 2023-12-17 13:45:42   
 
   label  
 0     装  
 1     卸  
 2     装  
 3     卸  
 4     装  ,
 (56, 6))

In [6]:
# 定义读取，处理数据函数
def fix_data(inputDF: pd.DataFrame) -> pd.DataFrame:
    # 创建一个新的空列用于存储结果

    inputDF.insert(0, "label", "-")

    # 对df1中的每一行遍历，并查找df2中符合条件的记录
    for index, row in inputDF.iterrows():
        condition = (dfstate["start"] <= row["时间"]) & (row["时间"] <= dfstate["end"])
        match = dfstate[condition]

        if not match.empty:
            # 如果找到了匹配项，则将df2的'state'赋值给df1的新列
            inputDF.at[index, "label"] = match["label"].values[0]
        else:
            # 如果没有找到匹配项，则保持原样（这里已经初始化为-）
            pass

    # inputDF.insert(1,'state',0)
    # inputDF["state"] = inputDF["label"].astype("category").cat.codes
    return inputDF


def read_data(file_path: str) -> pd.DataFrame:
    return fix_data(
        pd.read_excel(file_path, engine="openpyxl", parse_dates=["时间", "轨迹时间"])
    )


# 读取数据
df1 = read_data("./data/data_17.xlsx")
df21 = read_data("./data/data_21.xlsx")
df22 = read_data("./data/data_22.xlsx")
df23 = read_data("./data/data_23.xlsx")

# df1=pd.read_csv('data.csv')

# df1=df1.iloc[:,2:]
# df1.tail(),'rows',len(df1)

In [7]:
# df1.info() 
display(df1.shape,df21.shape,df22.shape,df23.shape)
# len(df1.query('state != 0')),len(df21.query('state != 0')),len(df22.query('state != 0')),len(df23.query('state != 0'))
# df1.head()
# display( df1.query('state != 0'),len(df21.query('state != 0')),len(df22.query('state != 0')),len(df23.query('state != 0')))
df1.dtypes

(16530, 29)

(24964, 25)

(27913, 25)

(27942, 29)

label                 object
时间            datetime64[ns]
原始重量                   int64
稳定重量                   int64
轴重                     int64
su_ad                  int64
wpu_x                  int64
wpu_y                  int64
wpu_z                  int64
su_x                   int64
su_y                   int64
su_z                   int64
原始重量_diff              int64
稳定重量_diff              int64
轴重_diff                int64
su_ad_diff             int64
wpu_x_diff             int64
wpu_y_diff             int64
wpu_z_diff             int64
su_x_diff              int64
su_y_diff              int64
su_z_diff              int64
轨迹时间          datetime64[ns]
速度                     int64
估计重量                   int64
原始重量加速度              float64
稳定重量加速度              float64
原始重量加速度_二阶           float64
稳定重量加速度_二阶           float64
dtype: object

In [24]:
from sklearn.preprocessing import StandardScaler, LabelEncoder

# 使用StandardScaler或者MinMaxScaler进行标准化处理。
# 创建标准化器对象
scaler = StandardScaler()
# （label）标准化
le = LabelEncoder()

number_types = [np.number]
label_types = ["object"]
time_types = ["datetime64[ns]"]
numeric_cols = df1.select_dtypes(include=number_types).columns.tolist()
label_cols = df1.select_dtypes(include=label_types).columns.tolist()
time_cols = df1.select_dtypes(include=time_types).columns.tolist()


def standard_scaler(inpudf: pd.DataFrame) -> pd.DataFrame:
    """对df进行标准化处理"""

    # 数值型数据标准化： 使用StandardScaler或者MinMaxScaler进行标准化处理。
    inpudf[numeric_cols] = scaler.fit_transform(inpudf[numeric_cols])

    # 类别数据（label）标准化： 类别数据通常不做标准化，而是进行编码，比如One-Hot编码或者Label Encoding。
    for col in label_cols:
        inpudf[col] = le.fit_transform(inpudf[col])

    # 时间数据标准化： 时间数据通常不进行标准化，而是转换成可以比较的形式，如Unix时间戳或时间段差。
    # 将所有时间列转换为Unix时间戳（单位为秒）
    for col in time_cols:
        inpudf[col] = (
            inpudf[col].astype(np.int64) / 1e9
        )  # 默认pandas的datetime64[ns]转换为Unix时间戳（秒）

    return inpudf


# 反标准化


def inverse_standard_scaler(inpudf: pd.DataFrame) -> pd.DataFrame:
    """对df中的数值列进行反标准化处理"""

    # 数值型数据标准化-
    inpudf[numeric_cols] = scaler.inverse_transform(inpudf[numeric_cols])

    # 类别数据（label）标准化-
    for col in label_cols:
        inpudf[col] = le.inverse_transform(inpudf[col])

    # 时间数据标准化-
    for col in time_cols:
        inpudf[col] = pd.to_datetime(
            inpudf[col] * 1e9
        )  # 默认pandas的datetime64[ns]转换为Unix时间戳（秒）

    return inpudf

In [27]:
df0 = standard_scaler(df1)

df1 = inverse_standard_scaler(df0)
display(df1.head())
display(df0.head())

,label,时间,原始重量,稳定重量,轴重,su_ad,wpu_x,wpu_y,wpu_z,su_x,...,su_x_diff,su_y_diff,su_z_diff,轨迹时间,速度,估计重量,原始重量加速度,稳定重量加速度,原始重量加速度_二阶,稳定重量加速度_二阶
0,-0.179119,-1.755540,-0.240046,-0.181312,-0.260416,0.260428,0.597393,0.644412,-0.419829,-0.818592,...,-0.000146,-0.000074,0.000057,-1.752945,-0.425997,-0.428828,0.002292,0.001302,-0.000001,7.747131e-18
1,-0.179119,-1.755339,-0.247974,-0.181312,-0.259641,0.259644,0.576117,0.644984,-0.399497,-0.818592,...,-0.000146,-0.000074,0.000057,-1.752945,-0.425997,-0.428828,0.002292,0.001302,-0.000001,7.747131e-18
2,-0.179119,-1.755072,-0.276288,-0.181312,-0.268239,0.268275,0.586755,0.636406,-0.409286,-0.818592,...,-0.000146,0.004801,0.003800,-1.752279,-0.425997,-0.428828,0.002292,0.001302,-0.000001,7.747131e-18
3,-0.179119,-1.754938,-0.258946,-0.181312,-0.261967,0.261998,0.586755,0.637550,-0.409286,-0.818592,...,-0.000146,0.005776,0.004548,-1.752279,-0.425997,-0.428828,0.002292,0.001302,-0.000001,7.747131e-18
4,-0.179119,-1.754737,-0.266874,-0.181312,-0.267464,0.267491,0.585936,0.637550,-0.408533,-0.812777,...,0.009147,-0.005924,-0.008176,-1.752279,-0.425997,-0.428828,0.002292,0.001302,-0.000001,7.747131e-18


,label,时间,原始重量,稳定重量,轴重,su_ad,wpu_x,wpu_y,wpu_z,su_x,...,su_x_diff,su_y_diff,su_z_diff,轨迹时间,速度,估计重量,原始重量加速度,稳定重量加速度,原始重量加速度_二阶,稳定重量加速度_二阶
0,-0.179119,-1.755540,-0.240046,-0.181312,-0.260416,0.260428,0.597393,0.644412,-0.419829,-0.818592,...,-0.000146,-0.000074,0.000057,-1.752945,-0.425997,-0.428828,0.002292,0.001302,-0.000001,7.747131e-18
1,-0.179119,-1.755339,-0.247974,-0.181312,-0.259641,0.259644,0.576117,0.644984,-0.399497,-0.818592,...,-0.000146,-0.000074,0.000057,-1.752945,-0.425997,-0.428828,0.002292,0.001302,-0.000001,7.747131e-18
2,-0.179119,-1.755072,-0.276288,-0.181312,-0.268239,0.268275,0.586755,0.636406,-0.409286,-0.818592,...,-0.000146,0.004801,0.003800,-1.752279,-0.425997,-0.428828,0.002292,0.001302,-0.000001,7.747131e-18
3,-0.179119,-1.754938,-0.258946,-0.181312,-0.261967,0.261998,0.586755,0.637550,-0.409286,-0.818592,...,-0.000146,0.005776,0.004548,-1.752279,-0.425997,-0.428828,0.002292,0.001302,-0.000001,7.747131e-18
4,-0.179119,-1.754737,-0.266874,-0.181312,-0.267464,0.267491,0.585936,0.637550,-0.408533,-0.812777,...,0.009147,-0.005924,-0.008176,-1.752279,-0.425997,-0.428828,0.002292,0.001302,-0.000001,7.747131e-18


In [ ]:
#进行数据归一化
from sklearn import preprocessing

# 初始化 MinMaxScaler 对象
min_max_scaler = preprocessing.MinMaxScaler()

# df0=min_max_scaler.fit_transform(df1)
# df = pd.DataFrame(df0, columns=df1.columns)
# input_size=len(df.iloc[1,:])


# 假设我们有一个DataFrame df，并且希望对其中的所有数值列进行标准化
# 先提取出所有数值列
numeric_features = df1.select_dtypes(include=[np.number]).columns.tolist()

# 使用 fit() 方法计算最小值和最大值（训练集上）
# min_max_scaler.fit(df1[numeric_features])
df0=min_max_scaler.fit_transform(df1[numeric_features])
# 使用 transform() 方法对数据进行标准化
inpudf = pd.DataFrame(df0, columns=numeric_features, index=df1.index)
input_size=len(inpudf.iloc[1,:])
display(df1.head())
display(inpudf.head())
display(input_size,inpudf.shape)

In [ ]:
#设置LSTM的时间窗等参数
window=5
lstm_units = 32
dropout = 0.01
epoch=60

#构建lstm输入
stock=inpudf
seq_len=window
amount_of_features = len(stock.columns)#有几列
data = stock.values #pd.DataFrame(stock) 表格转化为矩阵
sequence_length = seq_len + 1#序列长度
result = []
for index in range(len(data) - sequence_length):#循环数据长度-sequence_length次
    result.append(data[index: index + sequence_length])#第i行到i+sequence_length
result = np.array(result)#得到样本，样本形式为6天*3特征
row = round(0.9 * result.shape[0])#划分训练集测试集
train = result[:int(row), :]
x_train = train[:, :-1]
y_train = train[:, -1][:,-1]
x_test = result[int(row):, :-1]
y_test = result[int(row):, -1][:,-1]
#reshape成 6天*3特征
X_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], amount_of_features))
X_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], amount_of_features))  
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

inpudf.values[3:5]

In [ ]:
#建立LSTM模型 训练
inputs=Input(shape=(window, input_size))
model=Conv1D(filters = lstm_units, kernel_size = 1, activation = 'sigmoid')(inputs)#卷积层
model=MaxPooling1D(pool_size = window)(model)#池化层
model=Dropout(dropout)(model)#droupout层
model=Bidirectional(LSTM(lstm_units, activation='tanh'), name='bilstm')(model)#双向LSTM层
attention=Dense(lstm_units*2, activation='sigmoid', name='attention_vec')(model)#求解Attention权重
model=Multiply()([model, attention])#attention与LSTM对应数值相乘
outputs = Dense(1, activation='tanh')(model)
model = Model(inputs=inputs, outputs=outputs)
model.compile(loss='mse',optimizer='adam',metrics=['accuracy'])
model.summary()#展示模型结构

In [ ]:
history=model.fit(X_train, y_train, nb_epoch = epoch, batch_size = 256,shuffle=False,validation_data=(X_test, y_test)) #训练模型epoch次

In [ ]:
#迭代图像
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs_range = range(epoch)
plt.plot(epochs_range, loss, label='Train Loss')
plt.plot(epochs_range, val_loss, label='Test Loss')
plt.legend(loc='upper right')
plt.title('Train and Val Loss')
plt.show()

In [ ]:
#在训练集上的拟合结果
y_train_predict=model.predict(X_train)
y_train_predict=y_train_predict[:,0]
draw=pd.concat([pd.DataFrame(y_train),pd.DataFrame(y_train_predict)],axis=1)
draw.iloc[200:500,0].plot(figsize=(12,6))
draw.iloc[200:500,1].plot(figsize=(12,6))
plt.legend(('real', 'predict'),fontsize='15')
plt.title("Train Data",fontsize='30') #添加标题

In [ ]:
#在测试集上的预测
y_test_predict=model.predict(X_test)
y_test_predict=y_test_predict[:,0]
draw=pd.concat([pd.DataFrame(y_test),pd.DataFrame(y_test_predict)],axis=1);
draw.iloc[200:500,0].plot(figsize=(12,6))
draw.iloc[200:500,1].plot(figsize=(12,6))
plt.legend(('real', 'predict'),loc='upper right',fontsize='15')
plt.title("Test Data",fontsize='30') #添加标题

In [ ]:
#输出结果
def mape(y_true, y_pred):
    return np.mean(np.abs((y_pred - y_true) / y_true)) * 100
def up_down_accuracy(y_true, y_pred):
    y_var_test=y_true[1:]-y_true[:len(y_true)-1]#实际涨跌
    y_var_predict=y_pred[1:]-y_pred[:len(y_pred)-1]#原始涨跌
    txt=np.zeros(len(y_var_test))
    for i in range(len(y_var_test-1)):#计算数量
        txt[i]=np.sign(y_var_test[i])==np.sign(y_var_predict[i])
    result=sum(txt)/len(txt)
    return result
print('训练集上的MAE/MSE/MAPE/涨跌准确率')
print(mean_absolute_error(y_train_predict, y_train))
print(mean_squared_error(y_train_predict, y_train) )
print(mape(y_train_predict, y_train) )
print(up_down_accuracy(y_train_predict,y_train))
print('测试集上的MAE/MSE/MAPE/涨跌准确率')
print(mean_absolute_error(y_test_predict, y_test))
print(mean_squared_error(y_test_predict, y_test) )
print(mape(y_test_predict,  y_test) )
print(up_down_accuracy(y_test_predict,y_test))